In [6]:
import numpy as np
import random

In [7]:
def encode(data, poly):
    return np.polymul(data, poly) % 2

def add_random_errors(vector, count):
    size = len(vector)
    indices = random.sample(range(size), count)
    for idx in indices:
        vector[idx] ^= 1
    return vector, indices

def add_error_burst(vector, burst_size):
    size = len(vector)
    start = random.randint(0, size - burst_size)
    for i in range(burst_size):
        vector[(start + i) % size] ^= 1
    return vector, list(range(start, start + burst_size))

def check_burst(error, max_burst):
    error = np.trim_zeros(error, 'f')
    error = np.trim_zeros(error, 'b')
    return 0 < len(error) <= max_burst

def decode(vector, poly, threshold, burst_mode):
    length = len(vector)
    syndrome = np.polydiv(vector, poly)[1] % 2
    for shift in range(length):
        error_poly = np.zeros(length, dtype=int)
        error_poly[length - shift - 1] = 1
        product = np.polymul(syndrome, error_poly) % 2
        reduced_syndrome = np.polydiv(product, poly)[1] % 2
        if burst_mode:
            if check_burst(reduced_syndrome, threshold):
                corrected = np.polyadd(np.polymul(np.roll(np.eye(1, length, shift)[0], -1), reduced_syndrome) % 2, vector) % 2
                return np.array(np.polydiv(corrected, poly)[0] % 2).astype(int), shift
        else:
            if sum(reduced_syndrome) <= threshold:
                corrected = np.polyadd(np.polymul(np.roll(np.eye(1, length, shift)[0], -1), reduced_syndrome) % 2, vector) % 2
                return np.array(np.polydiv(corrected, poly)[0] % 2).astype(int), shift
    return None, None

In [8]:
poly = np.array([1, 1, 0, 1])
threshold = 1
for errors in range(1, 4):
    message = np.array([1, 0, 1, 0])
    print(f"Original message: {message}")
    encoded = encode(message, poly)
    print(f"Encoded message: {encoded}")
    corrupted, error_positions = add_random_errors(encoded.copy(), errors)
    print(f"Corrupted message: {corrupted}, Errors introduced at: {error_positions}")
    decoded, correction_shift = decode(corrupted, poly, threshold, burst_mode=False)
    print(f"Decoded message: {decoded}")
    print(f"Correction applied at position: {correction_shift}")
    if np.array_equal(message, decoded):
        print("Success: Decoded message matches the original.\n")
    else:
        print("Failure: Decoded message does not match the original.\n")

Original message: [1 0 1 0]
Encoded message: [1 1 1 0 0 1 0]
Corrupted message: [0 1 1 0 0 1 0], Errors introduced at: [0]
Decoded message: [1 0 1 0]
Correction applied at position: 1
Success: Decoded message matches the original.

Original message: [1 0 1 0]
Encoded message: [1 1 1 0 0 1 0]
Corrupted message: [1 1 1 0 1 0 0], Errors introduced at: [5, 4]
Decoded message: [0 1 0 0]
Correction applied at position: 1
Failure: Decoded message does not match the original.

Original message: [1 0 1 0]
Encoded message: [1 1 1 0 0 1 0]
Corrupted message: [1 1 0 0 0 0 1], Errors introduced at: [6, 5, 2]
Decoded message: [1 0 0 1]
Correction applied at position: 0
Failure: Decoded message does not match the original.



In [9]:
poly = np.array([1, 0, 0, 1, 1, 1, 1])
threshold = 3
for burst in range(1, 5):
    message = np.array([1, 1, 0, 0, 0, 1, 0, 0, 0])
    print(f"Original message: {message}")
    encoded = encode(message, poly)
    print(f"Encoded message: {encoded}")
    corrupted, error_positions = add_error_burst(encoded.copy(), burst)
    print(f"Corrupted message: {corrupted}, Error burst positions: {error_positions}")
    decoded, correction_shift = decode(corrupted, poly, threshold, burst_mode=True)
    print(f"Decoded message: {decoded}")
    print(f"Correction applied starting near position: {correction_shift}")
    if np.array_equal(message, decoded):
        print("Success: Decoded message matches the original.\n")
    else:
        print("Failure: Decoded message does not match the original.\n")

Original message: [1 1 0 0 0 1 0 0 0]
Encoded message: [1 1 0 1 0 1 0 1 1 1 1 1 0 0 0]
Corrupted message: [1 0 0 1 0 1 0 1 1 1 1 1 0 0 0], Error burst positions: [1]
Decoded message: [1 1 0 0 0 1 0 0 0]
Correction applied starting near position: 2
Success: Decoded message matches the original.

Original message: [1 1 0 0 0 1 0 0 0]
Encoded message: [1 1 0 1 0 1 0 1 1 1 1 1 0 0 0]
Corrupted message: [1 1 0 1 0 1 0 1 1 0 0 1 0 0 0], Error burst positions: [9, 10]
Decoded message: [1 1 0 0 0 1 0 0 0]
Correction applied starting near position: 0
Success: Decoded message matches the original.

Original message: [1 1 0 0 0 1 0 0 0]
Encoded message: [1 1 0 1 0 1 0 1 1 1 1 1 0 0 0]
Corrupted message: [1 0 1 0 0 1 0 1 1 1 1 1 0 0 0], Error burst positions: [1, 2, 3]
Decoded message: [1 1 0 0 0 1 0 0 0]
Correction applied starting near position: 4
Success: Decoded message matches the original.

Original message: [1 1 0 0 0 1 0 0 0]
Encoded message: [1 1 0 1 0 1 0 1 1 1 1 1 0 0 0]
Corrupted messa